In [2]:
import random
import torch
torch.cuda.set_device(0)
import cv2
import numpy as np
from tools.run_infinity import *

model_path='/mnt/bn/foundation-vision/hanjian.thu123/vgpt/checkpoints/hj_1217_infinity_2b_release_bsc03_data_80m/slim-ckpt-giter030K-ep0-iter30000-last.pth'
vae_path='/mnt/bn/foundation-vision/binyan.master/video_vae_results/cvpr_500k_64gpu_f16_bsq_dense_keep_last_rm_res_detach_lecam_zero_disc128_d2_dim32_hqv4_ms_dense_keep_first_rdn_short/checkpoints/model_step_199999.ckpt'
text_encoder_ckpt='/mnt/bn/foundation-vision/hanjian.thu123/data/flan-t5-xl'
# text_encoder_ckpt = '/mnt/bn/foundation-vision/hanjian.thu123/data/flan-t5-xl-official'
args=argparse.Namespace(
    pn='1M',
    model_path=model_path,
    cfg_insertion_layer=0,
    vae_type=32,
    vae_path=vae_path,
    add_lvl_embeding_only_first_block=1,
    use_bit_label=1,
    model_type='infinity_2b',
    rope2d_each_sa_layer=1,
    rope2d_normalized_by_hw=2,
    use_scale_schedule_embedding=0,
    sampling_per_bits=1,
    text_encoder_ckpt=text_encoder_ckpt,
    text_channels=2048,
    apply_spatial_patchify=0,
    h_div_w_template=1.000,
    use_flex_attn=0,
    cache_dir='/dev/shm',
    checkpoint_type='torch',
    seed=0,
    bf16=1,
    save_file='tmp.jpg'
)

/usr/local/lib/python3.9/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.9/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/tiger/.local/lib/python3.9/site-packages/bytedmetrics/__init__.py:10: UserWarning: bytedmetrics is renamed to bytedance.metrics, please using `bytedance.metrics` instead of `bytedmetrics`
  warnings.warn("bytedmetrics is renamed to bytedance.metrics, please using `bytedance.metrics` instead of `bytedmetrics`")
/usr/local/lib/python3.9/dist-packages/flash_attn-2.5.8-py3.9-linux-x86_64.egg/flash_attn/ops/fused_dense.py:30: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `

In [3]:
# load text encoder
text_tokenizer, text_encoder = load_tokenizer(t5_path=args.text_encoder_ckpt)
# load vae
vae = load_visual_tokenizer(args)
# load infinity
infinity = load_transformer(vae, args)

[Loading tokenizer and text encoder]


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

model_path: /mnt/bn/foundation-vision/hanjian.thu123/vgpt/checkpoints/hj_1217_infinity_2b_release_bsc03_data_80m/slim-ckpt-giter030K-ep0-iter30000-last.pth, slim_model_path: /mnt/bn/foundation-vision/hanjian.thu123/vgpt/checkpoints/hj_1217_infinity_2b_release_bsc03_data_80m/slim-ckpt-giter030K-ep0-iter30000-last.pth
local_model_path: /dev/shm/tmp/_mnt_bn_foundation-vision_hanjian.thu123_vgpt_checkpoints_hj_1217_infinity_2b_release_bsc03_data_80m_slim-ckpt-giter030K-ep0-iter30000-last.pth, local_slim_model_path: /dev/shm/tmp/_mnt_bn_foundation-vision_hanjian.thu123_vgpt_checkpoints_hj_1217_infinity_2b_release_bsc03_data_80m_slim-ckpt-giter030K-ep0-iter30000-last.pth
load checkpoint from /dev/shm/tmp/_mnt_bn_foundation-vision_hanjian.thu123_vgpt_checkpoints_hj_1217_infinity_2b_release_bsc03_data_80m_slim-ckpt-giter030K-ep0-iter30000-last.pth
[Loading Infinity]
self.codebook_dim: 32, self.add_lvl_embeding_only_first_block: 1,             self.use_bit_label: 1, self.rope2d_each_sa_layer: 1

/mnt/bn/foundation-vision/hanjian.thu123/infinity/speedup/vgpt/tools/run_infinity.py:178: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True, dtype=torch.bfloat16, cache_enabled=True), torch.no_grad():


self.num_blocks_in_a_chunk=4, depth=32, block_chunks=8

[constructor]  ==== customized_flash_attn=False (using_flash=0/32), fused_mlp=False (fused_mlp=0/32) ==== 
    [Infinity config ] embed_dim=2048, num_heads=16, depth=32, mlp_ratio=4, swiglu=False num_blocks_in_a_chunk=4
    [drop ratios] drop_rate=0.0, drop_path_rate=0.1 (tensor([0.0000, 0.0032, 0.0065, 0.0097, 0.0129, 0.0161, 0.0194, 0.0226, 0.0258,
        0.0290, 0.0323, 0.0355, 0.0387, 0.0419, 0.0452, 0.0484, 0.0516, 0.0548,
        0.0581, 0.0613, 0.0645, 0.0677, 0.0710, 0.0742, 0.0774, 0.0806, 0.0839,
        0.0871, 0.0903, 0.0935, 0.0968, 0.1000]))

[you selected Infinity with model_kwargs={'depth': 32, 'embed_dim': 2048, 'num_heads': 16, 'drop_path_rate': 0.1, 'mlp_ratio': 4, 'block_chunks': 8}] model size: 2.20B, bf16=1
[Load Infinity weights]


/mnt/bn/foundation-vision/hanjian.thu123/infinity/speedup/vgpt/tools/run_infinity.py:210: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, m

<All keys matched successfully>


In [112]:
# prompt = """a woman lying on the beach"""
prompt = """a group of students in a class"""
cfg = 3
tau = 0.5
h_div_w = 1/1 # aspect ratio, height:width
seed = random.randint(0, 10000)
enable_positive_prompt=0

h_div_w_template_ = h_div_w_templates[np.argmin(np.abs(h_div_w_templates-h_div_w))]
scale_schedule = dynamic_resolution_h_w[h_div_w_template_][args.pn]['scales']
scale_schedule = [(1, h, w) for (_, h, w) in scale_schedule]
generated_image = gen_one_img(
    infinity,
    vae,
    text_tokenizer,
    text_encoder,
    prompt,
    g_seed=seed,
    gt_leak=0,
    gt_ls_Bl=None,
    cfg_list=cfg,
    tau_list=tau,
    scale_schedule=scale_schedule,
    cfg_insertion_layer=[args.cfg_insertion_layer],
    vae_type=args.vae_type,
    sampling_per_bits=args.sampling_per_bits,
    enable_positive_prompt=enable_positive_prompt,
)
args.save_file = 'ipynb_tmp.jpg'
os.makedirs(osp.dirname(osp.abspath(args.save_file)), exist_ok=True)
cv2.imwrite(args.save_file, generated_image.cpu().numpy())
print(f'Save to {osp.abspath(args.save_file)}')

prompt=a group of students in a class
cfg: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], tau: [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]


cost: 0.8944292068481445, infinity cost=0.8735930919647217
Save to /mnt/bn/foundation-vision/hanjian.thu123/infinity/speedup/vgpt/tools/ipynb_tmp.jpg
